Use this technique to impersonate a user before connecting to a service that uses Microsoft AD authentication

In [ ]:
import pandas
import json
import win32security, win32con, win32.win32api , win32 ,win32.win32security
import pyodbc

# Need this to personate a user
class Impersonate:
    def __init__(self, domain, login, password):
        self.domain = domain
        self.login = login
        self.password = password
 
    def logon(self):
        self.handle = win32.win32security.LogonUser(self.login, self.domain,self.password,win32con.LOGON32_LOGON_INTERACTIVE,win32con.LOGON32_PROVIDER_DEFAULT)
        win32.win32security.ImpersonateLoggedOnUser(self.handle)
 
    def logoff(self):
        win32security.RevertToSelf() # terminates impersonation
        self.handle.Close() # guarantees cleanup
 
a = Impersonate('your domain', 'your user','your password')
# Logging in
a.logon()
 
#SQL Server
sqlserver = 'tcp:your sql server' 
sqlconnection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+sqlserver+';trusted_connection=yes')
sqlquery = 'SELECT GETDATE()'
try:
    
    print(f'Debug: {sqlquery}')
    pandas.read_sql_query(sqlquery, sqlconnection)
    
except Exception as err:
    print(f"Error: {err}")
finally:
    sqlconnection.close()
    # since we done on the server we can logoff
    Impersonate.logoff(a)

print('Done!')
